In [ ]:
ROOT_DIR = "/content/drive/MyDrive/Mask_RCNN/"

from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Just in case the pip mrcnn package is installed, uninstall it.
# We want to use the M R-CNN version in our drive.
!pip uninstall mrcnn

# Uninstall the default packages Colab uses so we can use the versions specified in requirements.txt.

!pip uninstall keras -y
!pip uninstall keras-nightly -y
!pip uninstall keras-Preprocessing -y
!pip uninstall keras-vis -y
!pip uninstall tensorflow -y
!pip uninstall kerassurgeon -y
!pip uninstall h5py -y

Found existing installation: Keras 2.0.8
Uninstalling Keras-2.0.8:
  Successfully uninstalled Keras-2.0.8
Found existing installation: Keras-Preprocessing 1.1.2
Uninstalling Keras-Preprocessing-1.1.2:
  Successfully uninstalled Keras-Preprocessing-1.1.2
Found existing installation: kerassurgeon 0.2.0
Uninstalling kerassurgeon-0.2.0:
  Successfully uninstalled kerassurgeon-0.2.0
Found existing installation: h5py 2.10.0
Uninstalling h5py-2.10.0:
  Successfully uninstalled h5py-2.10.0


In [ ]:
!pip install -r /content/drive/MyDrive/Mask_RCNN/requirements.txt

  Using cached Keras-2.0.8-py2.py3-none-any.whl (276 kB)
  Using cached h5py-2.10.0-cp37-cp37m-manylinux1_x86_64.whl (2.9 MB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)


In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import time
import copy

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/balloon/"))  # To find local version
import balloon

Using TensorFlow backend.


ImportError: ignored

In [ ]:
#python3 coco.py train --dataset=/path/to/coco/ --model=coco
#python3 balloon.py train --dataset=/path/to/balloon/dataset --weights=coco

DATASET_PATH = os.path.join(ROOT_DIR, "datasets/balloon")

"""Train the model."""
# Training dataset.
dataset_train = balloon.BalloonDataset()
dataset_train.load_balloon(DATASET_PATH, "train")
dataset_train.prepare()

# Validation dataset
dataset_val = balloon.BalloonDataset()
dataset_val.load_balloon(DATASET_PATH, "val")
dataset_val.prepare()

In [ ]:
# Create model object in inference mode.

from mrcnn.config import Config

MODEL_DIR = os.path.join(ROOT_DIR, "logs")

class BalloonConfig(Config):
    """Configuration for training on the toy  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "balloon"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + balloon

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 50

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9

config = BalloonConfig()

model = modellib.MaskRCNN(mode="training", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
model.load_weights(COCO_MODEL_PATH, by_name=True, exclude=[
    "mrcnn_class_logits", "mrcnn_bbox_fc",
    "mrcnn_bbox", "mrcnn_mask"])

In [ ]:
# Prune layers that contain the following terms
layers_to_prune = [
  "bn"
]

def should_prune_layer(layer):
  # If not trainable, don't prune this layer.
  if not layer.trainable:
    return False

  # Only prune if this layer name matches a pattern in the list of layers to prune.
  for layer_pattern in layers_to_prune:
    if layer_pattern in layer.name:
      return True

  return False


def prune(model, threshold=0.1):

    layers_processed = 0
    for layer in model.keras_model.layers: 
      layers_processed += 1
      if not should_prune_layer(layer):
        continue
        
      # Find the weights that are the closest 10% to 0 so we can prune them.
      threshold_percentage = 10
      weights = np.array(layer.get_weights())
      flattened_weights = np.absolute(weights.flatten())
      weight_threshold = np.percentile(flattened_weights, threshold_percentage)

      def prune(value):
        if ( value > 0 and value < weight_threshold ) or ( value < 0 and value > -weight_threshold ):
          return 0
        return value
      
      prune_vectorized = np.vectorize(prune)
      pruned_weights = prune_vectorized(weights)

      layer.set_weights(pruned_weights)
    print(layers_processed)

In [ ]:
prune(model)

1
2
3
4
5
6


In [ ]:
from mrcnn.model import data_generator

val_generator = data_generator(dataset_val, config, shuffle=True)
model.keras_model.evaluate_generator(val_generator, 1)

[2.1098484992980957,
 0.05976346135139465,
 1.1851619482040405,
 0.20026709139347076,
 0.42458948493003845,
 0.24006500840187073]

In [ ]:
print("Training network heads")
model.train(dataset_train, dataset_val, learning_rate=config.LEARNING_RATE, epochs= 4, layers='heads')

Training network heads

Starting at epoch 2. LR=0.001

Checkpoint Path: /content/drive/MyDrive/Mask_RCNN/logs/balloon20211128T1918/mask_rcnn_balloon_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv

In [ ]:
from mrcnn.model import data_generator

val_generator = data_generator(dataset_val, config, shuffle=True)
model.keras_model.evaluate_generator(val_generator, 20)

[0.6531634278595447,
 0.030853548372397198,
 0.21443604012019932,
 0.10772452260134742,
 0.18662750199437142,
 0.1135202756151557]

In [ ]:
import tensorflow_model_optimization as tfmot

/usr/local/lib/python3.7/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

AttributeError: ignored